## Лабораторная работа №2

### 1,2. Загружаем все необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import pandas
import numpy
import pickle
from sklearn.preprocessing import StandardScaler, TargetEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import mlflow
from mlflow.models import infer_signature
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer, QuantileTransformer
import optuna
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, r2_score
from mlxtend.feature_selection import SequentialFeatureSelector
import numpy as np

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 3. Загружаем очещенный датасет из лр1

In [2]:
with open("../data/new_clean_dataset.pkl", "rb") as f:
    df = pickle.load(f)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1935 entries, 1 to 1999
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   battery_power       1935 non-null   int64   
 1   blue                1935 non-null   category
 2   clock_speed         1935 non-null   float64 
 3   dual_sim            1935 non-null   category
 4   fc                  1935 non-null   int64   
 5   four_g              1935 non-null   category
 6   int_memory          1935 non-null   int64   
 7   m_dep               1935 non-null   float64 
 8   mobile_wt           1935 non-null   int64   
 9   n_cores             1935 non-null   int64   
 10  pc                  1935 non-null   int64   
 11  px_height           1935 non-null   int64   
 12  px_width            1935 non-null   int64   
 13  ram                 1935 non-null   int64   
 14  sc_h                1935 non-null   int64   
 15  sc_w                1935 non-null   int64  

#### Разрделяем датасет на две выборки: обучающую и тестовую в соотношении 25% и 75%

In [3]:
X = df.drop('price_range', axis=1)
y = df['price_range']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.75)

### 4. Создаём переменные, содержащие названия столбцов с числовыми и категориальными признаками

In [4]:
cat_features = ['blue', 'dual_sim', 'four_g', 'three_g', 
                'touch_screen', 'wifi', 'n_cores']

num_features = ['battery_power', 'clock_speed', 'fc', 'int_memory', 
                'm_dep', 'mobile_wt', 'pc', 'px_height', 'px_width', 
                'ram', 'sc_h', 'sc_w', 'talk_time', 'battery_efficiency', 'screen_size']

In [5]:
for col in cat_features:
    df[col] = df[col].astype('category')

cat_features_found = df.select_dtypes(include=['category']).columns.to_list()
num_features_found = df.select_dtypes(include=['number']).columns.to_list()

print("Значения типа number:", num_features)
print("Значения типа object:", cat_features)

Значения типа number: ['battery_power', 'clock_speed', 'fc', 'int_memory', 'm_dep', 'mobile_wt', 'pc', 'px_height', 'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'battery_efficiency', 'screen_size']
Значения типа object: ['blue', 'dual_sim', 'four_g', 'three_g', 'touch_screen', 'wifi', 'n_cores']


### 5. Создаём pipeline обработки признаков и обучения модели

In [6]:
s_scaler = StandardScaler()
l_encoder = TargetEncoder()
regressor = RandomForestRegressor(random_state=42)

#### Преобразования для числовых и категориальных признаков

In [7]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  
        ('cat', l_encoder, cat_features), 
    ],
    remainder='drop' )

### 6. Обучение baseline-модель и получение метрик качества на тестовой выборке

In [8]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor)])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['battery_power',
                                                   'clock_speed', 'fc',
                                                   'int_memory', 'm_dep',
                                                   'mobile_wt', 'pc',
                                                   'px_height', 'px_width',
                                                   'ram', 'sc_h', 'sc_w',
                                                   'talk_time',
                                                   'battery_efficiency',
                                                   'screen_size']),
                                                 ('cat', TargetEncoder(),
                                                  ['blue', 'dual_sim', 'four_g',
                                                   'three_g', 'touch_screen',
                                                   'wifi', 'n_cores'])])),
                ('model', RandomForestRegressor(random_state=42))])

In [9]:
predictions = pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)

{'MAE': 0.17923553719008264, 'MSE': 0.0772845041322314, 'R2': 0.9372080659538088}


### 7,8. Настройка доступа для подключения к mlflow

In [10]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

registry_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"
tracking_uri = f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}"

mlflow.set_tracking_uri(tracking_uri)   
mlflow.set_registry_uri(registry_uri)

### 9. Логируем baseline-модель в новый эксперимент. Регистрируем модель.

In [11]:
EXPERIMENT_NAME = "Mobile Classification 3.0"
RUN_NAME = "baseline model"
REGISTRY_MODEL_NAME = "price_range_prediction_model"

In [12]:
signature =  infer_signature(model_input = X_train.head(5))
input_example = X_train.head(5)

/Users/alexandr/MPEI/IIS/.venv/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [13]:
req_file = '../requirements.txt'
params_dict = pipeline.get_params()

#### Запускаем эксперимент, random_state=42

In [14]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id
    
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
  
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)

    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/11/27 05:15:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 05:15:47 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: matplotlib==3.10.6)
 - pandas (current: 2.2.2, required: pandas==2.3.2)
 - scikit-learn-1.7.2 (current: uninstalled, required: scikit-learn-1.7.2)
 - numpy (current: 2.0.2, required: numpy==1.26.4)
 - mlflow (current: 3.1.4, required: mlflow==2.16)
 - optuna-4.6.0 (current: uninstalled, required: optuna-4.6.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/11/27 05:15:47 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: m

🏃 View run baseline model at: http://127.0.0.1:5000/#/experiments/5/runs/d15b1bbfb506483497cb258b888620fb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


#### Запускаем эксперимент, random_state=43

In [15]:
regressor2 = RandomForestRegressor(random_state=43)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), 
                           ('model', regressor2)])

print(pipeline.fit(X_train, y_train))

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['battery_power',
                                                   'clock_speed', 'fc',
                                                   'int_memory', 'm_dep',
                                                   'mobile_wt', 'pc',
                                                   'px_height', 'px_width',
                                                   'ram', 'sc_h', 'sc_w',
                                                   'talk_time',
                                                   'battery_efficiency',
                                                   'screen_size']),
                                                 ('cat', TargetEncoder(),
                                                  ['blue', 'dual_sim', 'four_g',
                                                   'three_g', 'touch_screen',
                    

#### Значение метрик для random_state=43

In [16]:
predictions = pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)

{'MAE': 0.17944214876033057, 'MSE': 0.07958078512396694, 'R2': 0.9353423889179844}


### 10. С использованием библиотеки sklearn создали дополнительные признаки, обучили модель, залогировали ее.

In [17]:
X_train_fe_sklearn = X_train.copy()

pf = PolynomialFeatures(degree=2)

X_train_fe_sklearn

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,battery_efficiency,screen_size
690,1403,0,2.7,0,2,1,26,0.1,164,5,...,1251,3371,13,9,9,1,0,1,8.554878,15.811388
1425,1600,0,2.5,1,1,0,19,0.6,88,6,...,1713,1179,10,3,18,0,0,1,18.181818,10.440307
1926,1122,0,0.5,0,0,1,40,0.3,156,7,...,1163,1456,9,3,20,1,1,0,7.192308,9.486833
386,1138,0,1.4,0,12,1,44,0.2,187,8,...,1138,3475,19,3,3,1,0,0,6.085561,19.235384
1027,502,0,1.5,1,7,0,37,0.2,199,2,...,1810,1649,6,1,14,0,1,0,2.522613,6.082763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,1722,1,1.0,0,7,0,25,0.8,88,6,...,1638,2376,5,1,19,0,0,1,19.568182,5.099020
949,1584,1,1.3,0,6,0,41,0.6,135,4,...,1291,1561,10,5,17,0,0,1,11.733333,11.180340
256,601,1,1.4,0,4,1,4,0.1,154,4,...,765,532,12,8,10,1,1,1,3.902597,14.422205
1768,719,1,0.5,1,0,1,23,0.4,113,6,...,1727,3990,14,9,12,1,1,1,6.362832,16.643317


In [18]:
pf = PolynomialFeatures(degree=2)
qt = QuantileTransformer()
sp = SplineTransformer(n_knots=3, degree=3)

In [19]:
pf_pipeline = Pipeline(steps=[
    ('poly', pf),
    ('scale', StandardScaler())
])

#### Преобразуем числовые и категориальные признаки

In [20]:
preprocessor_sklearn = ColumnTransformer(
    transformers=[
        ('num', s_scaler, num_features),  
        ('cat', l_encoder, cat_features), 
        ('quantile', qt, num_features),
        ('poly', pf_pipeline, ['ram', 'battery_power']),
        ('spline', sp, ['ram'])
    ],
    remainder='drop',
    )

In [21]:
X_train_fe_sklearn[['ram','battery_power']] = X_train_fe_sklearn[['ram','battery_power']].astype('float64')
X_train_fe_sklearn[['ram','battery_power']] = X_train_fe_sklearn[['ram','battery_power']].astype('float64')

In [22]:
X_train_sklearn_raw = preprocessor_sklearn.fit_transform(X_train_fe_sklearn, y_train)
X_train_fe_sklearn = pd.DataFrame(X_train_sklearn_raw, columns=preprocessor_sklearn.get_feature_names_out())

In [23]:
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display (X_train_fe_sklearn)

,num__battery_power,num__clock_speed,num__fc,num__int_memory,num__m_dep,num__mobile_wt,num__pc,num__px_height,num__px_width,num__ram,num__sc_h,num__sc_w,num__talk_time,num__battery_efficiency,num__screen_size,cat__blue_0,cat__blue_1,cat__blue_2,cat__blue_3,cat__dual_sim_0,cat__dual_sim_1,cat__dual_sim_2,cat__dual_sim_3,cat__four_g_0,cat__four_g_1,cat__four_g_2,cat__four_g_3,cat__three_g_0,cat__three_g_1,cat__three_g_2,cat__three_g_3,cat__touch_screen_0,cat__touch_screen_1,cat__touch_screen_2,cat__touch_screen_3,cat__wifi_0,cat__wifi_1,cat__wifi_2,cat__wifi_3,cat__n_cores_0,cat__n_cores_1,cat__n_cores_2,cat__n_cores_3,quantile__battery_power,quantile__clock_speed,quantile__fc,quantile__int_memory,quantile__m_dep,quantile__mobile_wt,quantile__pc,quantile__px_height,quantile__px_width,quantile__ram,quantile__sc_h,quantile__sc_w,quantile__talk_time,quantile__battery_efficiency,quantile__screen_size,poly__1,poly__ram,poly__battery_power,poly__ram^2,poly__ram battery_power,poly__battery_power^2,spline__ram_sp_0,spline__ram_sp_1,spline__ram_sp_2,spline__ram_sp_3,spline__ram_sp_4
0,0.368369,1.421502,-0.538199,-0.324708,-1.383859,0.658525,-1.146780,-0.478066,-0.018028,1.172002,0.172334,0.742727,-0.377923,-0.208007,0.364298,0.259921,0.251654,0.258261,0.230162,0.247449,0.264481,0.237219,0.250850,0.261153,0.228558,0.247424,0.262862,0.247145,0.242572,0.256283,0.253999,0.239662,0.236061,0.261237,0.263037,0.242389,0.239002,0.252535,0.266071,0.235010,0.208303,0.274975,0.281641,0.602722,0.896897,0.406406,0.408408,0.000000,0.700701,0.175676,0.397898,0.501379,0.836760,0.540040,0.762262,0.395395,0.469279,0.600601,0.0,1.172002,0.368369,1.224123,1.213414,0.210262,0.000000,6.272346e-03,0.371558,0.573182,0.048988
1,0.812460,1.177840,-0.766674,-0.712807,0.349249,-1.473830,-0.164792,0.361541,1.057063,-0.857307,-0.535374,-0.643676,1.261167,2.026750,-0.680322,0.259920,0.254961,0.248344,0.236773,0.262956,0.247412,0.233588,0.256042,0.252786,0.256497,0.252780,0.237937,0.267597,0.263894,0.215723,0.252771,0.237435,0.235639,0.253587,0.273333,0.235619,0.247465,0.247459,0.269454,0.229678,0.254079,0.229653,0.286541,0.734735,0.831331,0.294294,0.303804,0.604605,0.074074,0.448949,0.669670,0.804805,0.246232,0.341341,0.317317,0.861361,0.958382,0.311311,0.0,-0.857307,0.812460,-0.893389,-0.424647,0.739485,0.021680,4.833309e-01,0.474980,0.020009,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1449,-1.173551,-1.258780,-0.995149,-0.491036,-0.343994,-0.772398,-0.164792,-0.546142,1.089641,1.745060,0.408237,0.742727,0.168440,-0.716860,0.526100,0.242821,0.248208,0.248202,0.260768,0.262956,0.247412,0.233588,0.256042,0.250805,0.247595,0.244379,0.257220,0.246919,0.248040,0.258126,0.246915,0.264878,0.266531,0.243385,0.225201,0.235619,0.247465,0.247459,0.269454,0.229678,0.254079,0.229653,0.286541,0.163163,0.000000,0.000000,0.364865,0.412913,0.283283,0.448949,0.369870,0.815816,0.997678,0.606106,0.762262,0.554555,0.262003,0.663664,0.0,1.745060,-1.173551,2.191512,0.141369,-1.088168,0.000000,1.302859e-08,0.168814,0.666648,0.164538
1450,-0.080231,1.299671,-0.995149,-1.211791,-0.690616,1.640530,-1.146780,-0.434951,0.519517,-0.352757,-0.299471,-0.874743,1.443289,-0.789421,-0.536393,0.242821,0.248208,0.248202,0.260768,0.240568,0.256006,0.262861,0.240563,0.252786,0.256497,0.252780,0.237937,0.246919,0.248040,0.258126,0.246915,0.237435,0.235639,0.253587,0.273333,0.268391,0.256133,0.249121,0.226352,0.248251,0.177711,0.290450,0.283417,0.481442,0.866867,0.000000,0.148148,0.320821,0.983984,0.175676,0.416416,0.659660,0.400400,0.400400,0.228729,0.916416,0.233224,0.351351,0.0,-0.352757,-0.080231,-0.557483,-0.315584,-0.253838,0.001665,2.925636e-01,0.625269,0.080502,0.000000


In [24]:
pipeline_sklearn = Pipeline(steps=[
    ('transform', preprocessor_sklearn),
    ('model', regressor)
])

model_sklearn = pipeline_sklearn.fit(X_train, y_train)

In [25]:
predictions = model_sklearn.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions)
metrics["MSE"] = mean_squared_error(y_test, predictions)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)

{'MAE': 0.15747933884297521, 'MSE': 0.06857644628099173, 'R2': 0.9442831685297393}


In [26]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)

if experiment is None:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name="sklearn_regressor", experiment_id=experiment_id) as run:
   
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(pipeline, 
                             artifact_path="models",
                             signature=signature,
                             input_example=input_example,
                             pip_requirements=req_file
                             )
    mlflow.log_metrics(metrics)

    mlflow.log_params(params_dict)

run = mlflow.get_run(run_id) 
assert (run.info.status =='FINISHED')

2025/11/27 05:16:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 05:16:18 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: matplotlib==3.10.6)
 - pandas (current: 2.2.2, required: pandas==2.3.2)
 - scikit-learn-1.7.2 (current: uninstalled, required: scikit-learn-1.7.2)
 - numpy (current: 2.0.2, required: numpy==1.26.4)
 - mlflow (current: 3.1.4, required: mlflow==2.16)
 - optuna-4.6.0 (current: uninstalled, required: optuna-4.6.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/11/27 05:16:18 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: m

🏃 View run sklearn_regressor at: http://127.0.0.1:5000/#/experiments/5/runs/0c7db02d13c740c78944d7835c964520
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


### 12. С использованием библиотеки mlxtend отобрал N наиболее важных признаков. N выбирается с учетом количества признаков на предыдущем шаге, ориентировочный диапазон - от 20% до 70%. Обучил модель, залогировал ее.

In [27]:
class ColumnExtractor(object):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        return X[:, self.cols]
    
    def fit(self, X, y=None):
        return self

In [28]:
sfs = SequentialFeatureSelector(
    RandomForestRegressor(random_state=43), 
    k_features=3,
    forward=True,
    floating=False,
    scoring='neg_mean_absolute_error',
    cv=2
)

sfs.fit(X_train_fe_sklearn, y_train)

sfs_cols = list(X_train_fe_sklearn.columns[list(sfs.k_feature_idx_)])
sfs_idx = list(sfs.k_feature_idx_)

print(sfs_cols)

with open('sfs_cols.txt', 'w+') as f:
    f.write(str(sfs_cols))
with open('sfs_idx.txt', 'w+') as f:
    f.write(str(sfs_idx))


sfs_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor_sklearn),
    ('sfs_extractor', ColumnExtractor(sfs_idx)),
    ('model', regressor)
])

sfs_pipeline.fit(X_train, y_train)


predictions_sfs = sfs_pipeline.predict(X_test)
metrics = {}
metrics["MAE"] = mean_absolute_error(y_test, predictions_sfs)
metrics["MSE"] = mean_squared_error(y_test, predictions_sfs)
metrics["R2"] = r2_score(y_test, predictions)

print(metrics)


experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = 'sequential_feature_selection'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    
   
    mlflow.sklearn.log_model(
        sfs_pipeline, 
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    mlflow.log_metrics(metrics)
    mlflow.log_artifact('sfs_cols.txt')
    mlflow.log_artifact('sfs_idx.txt')
    
    params = {
        'n_features_selected': len(sfs_cols),
        'forward': True,
        'floating': False,
        'scoring': 'neg_mean_absolute_error'
    }
    mlflow.log_params(params)
    
    model_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
  
    client = mlflow.tracking.MlflowClient()
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=model_version.version,
        description="Random Forest with Sequential Feature Selection"
    )

print(f"Model registered as version {model_version.version} of {REGISTRY_MODEL_NAME}")

2025/11/27 05:16:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 05:16:51 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: matplotlib==3.10.6)
 - pandas (current: 2.2.2, required: pandas==2.3.2)
 - scikit-learn-1.7.2 (current: uninstalled, required: scikit-learn-1.7.2)
 - numpy (current: 2.0.2, required: numpy==1.26.4)
 - mlflow (current: 3.1.4, required: mlflow==2.16)
 - optuna-4.6.0 (current: uninstalled, required: optuna-4.6.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/11/27 05:16:51 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: m

['num__px_width', 'poly__ram battery_power', 'spline__ram_sp_1']
{'MAE': 0.1577685950413223, 'MSE': 0.08687190082644626, 'R2': 0.9442831685297393}


Registered model 'price_range_prediction_model' already exists. Creating a new version of this model...
2025/11/27 05:16:51 WARNING mlflow.tracking._model_registry.fluent: Run with id d21b799d65a44682852fb1f139c1d10a has no artifacts at artifact path 'models', registering model based on models:/m-83afd6b277354e12b5bdfd9492800e7e instead
2025/11/27 05:16:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: price_range_prediction_model, version 4


🏃 View run sequential_feature_selection at: http://127.0.0.1:5000/#/experiments/5/runs/d21b799d65a44682852fb1f139c1d10a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
Model registered as version 4 of price_range_prediction_model


Created version '4' of model 'price_range_prediction_model'.


### 14. С помощью optuna настроить оптимальные параметры для модели, показывающей лучший результат. Обучить модель, залогировал ее, зарегистрировать очередную версию.

In [29]:
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 20),
        'n_estimators': trial.suggest_int('n_estimators', 10, 100),
        'max_features': trial.suggest_float('max_features', 0.1, 1.0),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10)
    }
    
    regressor_trial = RandomForestRegressor(**params, random_state=44)
    pipeline_trial = Pipeline(steps=[
        ('preprocessor', preprocessor_sklearn), 
        ('sfs_extractor', ColumnExtractor(sfs_idx)),  
        ('model', regressor_trial)
    ])  
    
    pipeline_trial.fit(X_train, y_train)
    predictions = pipeline_trial.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)

    return mae

study = optuna.create_study(direction='minimize')  
study.optimize(objective, n_trials=30)  

best_params = study.best_params
print("Лучшие параметры:", best_params)

regressor_optimized = RandomForestRegressor(**best_params, random_state=42)
sfs_optuna_pipeline = Pipeline(steps=[  
    ('preprocessor', preprocessor_sklearn), 
    ('sfs_extractor', ColumnExtractor(sfs_idx)),  
    ('model', regressor_optimized)
])

sfs_optuna_pipeline.fit(X_train, y_train)

predictions_optuna = sfs_optuna_pipeline.predict(X_test)
metrics = {
    "MAE": mean_absolute_error(y_test, predictions_optuna),
    "MSE": mean_squared_error(y_test, predictions_optuna),
    "R2": r2_score(y_test, predictions_optuna)
}

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
RUN_NAME = 'sfs_feature_selection_optimised_Optuna'

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id 
    mlflow.sklearn.log_model(
        sfs_optuna_pipeline,  
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    mlflow.log_metrics(metrics)
    mlflow.log_params(best_params)
    mlflow.log_artifact('sfs_cols.txt')  
    mlflow.log_artifact('sfs_idx.txt')   
    
    model_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
    client = mlflow.tracking.MlflowClient()
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=model_version.version,
        description="Optimized Random Forest with Optuna and SFS feature selection"
    )

print(f"Model registered as version {model_version.version} of {REGISTRY_MODEL_NAME}")

[I 2025-11-27 05:16:59,792] A new study created in memory with name: no-name-77d06a01-fa12-48de-97ba-f3d359499021
[I 2025-11-27 05:16:59,853] Trial 0 finished with value: 0.4428815423282054 and parameters: {'max_depth': 2, 'n_estimators': 36, 'max_features': 0.16224854989426166, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.4428815423282054.
[I 2025-11-27 05:16:59,897] Trial 1 finished with value: 0.1808564479968527 and parameters: {'max_depth': 15, 'n_estimators': 28, 'max_features': 0.13261159363898598, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.1808564479968527.
[I 2025-11-27 05:16:59,939] Trial 2 finished with value: 0.17794202606548318 and parameters: {'max_depth': 6, 'n_estimators': 23, 'max_features': 0.9100786092660511, 'min_samples_split': 6, 'min_samples_leaf': 5}. Best is trial 2 with value: 0.17794202606548318.
[I 2025-11-27 05:16:59,985] Trial 3 finished with value: 0.20814359916348835 and parameters: {'max

Лучшие параметры: {'max_depth': 18, 'n_estimators': 67, 'max_features': 0.8855328805678252, 'min_samples_split': 2, 'min_samples_leaf': 1}


Registered model 'price_range_prediction_model' already exists. Creating a new version of this model...
2025/11/27 05:17:02 WARNING mlflow.tracking._model_registry.fluent: Run with id cfe4b5027a35404ea079d57bc5fc1499 has no artifacts at artifact path 'models', registering model based on models:/m-a64703f3256e49d79ea3263e2f81a6ff instead
2025/11/27 05:17:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: price_range_prediction_model, version 5


🏃 View run sfs_feature_selection_optimised_Optuna at: http://127.0.0.1:5000/#/experiments/5/runs/cfe4b5027a35404ea079d57bc5fc1499
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5
Model registered as version 5 of price_range_prediction_model


Created version '5' of model 'price_range_prediction_model'.


### 16.Проанализировал все прогоны и выбрал модель, показывающую наилучшее качество. Обучил эту модель на всей выборке (а не только на train-части), залогировал ее. В реестре моделей установил ей тэг Production. Эту модель мы будем деплоить в следующей лабораторной работе. 

In [30]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])
best_run = runs.loc[runs['metrics.MAE'].idxmin()]
best_run_id = best_run['run_id']
best_mae = best_run['metrics.MAE']

print(f"Лучшая модель: Run ID {best_run_id}")
print(f"Лучший MAE: {best_mae}")
print(f"Метод: {best_run['tags.mlflow.runName']}")

Лучшая модель: Run ID 0c7db02d13c740c78944d7835c964520
Лучший MAE: 0.15747933884297521
Метод: sklearn_regressor


In [31]:
X_full = df.drop('price_range', axis=1)
y_full = df['price_range']

print(f"Размер всей выборки: {X_full.shape}")

Размер всей выборки: (1935, 22)


In [32]:
best_run = mlflow.get_run(best_run_id)
best_params = best_run.data.params

print("Параметры лучшей модели:", best_params)

# Функция для безопасного преобразования параметров
def get_param(params_dict, *keys, default, converter=int):
    """Безопасное получение параметра с конвертацией"""
    for key in keys:
        value = params_dict.get(key)
        if value is not None and value != 'None':
            try:
                return converter(value)
            except (ValueError, TypeError):
                continue
    return default

final_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features),  
            ('cat', TargetEncoder(), cat_features), 
            ('quantile', QuantileTransformer(), num_features),
            ('poly', Pipeline(steps=[
                ('poly', PolynomialFeatures(degree=2)),
                ('scale', StandardScaler())
            ]), ['ram', 'battery_power']),
            ('spline', SplineTransformer(n_knots=3), ['ram'])
        ],
        remainder='drop'
    )),
    ('sfs_extractor', ColumnExtractor(sfs_idx)),
    ('model', RandomForestRegressor(
        random_state=42,
        max_depth=get_param(best_params, 'model__max_depth', 'max_depth', default=10, converter=int),
        n_estimators=get_param(best_params, 'model__n_estimators', 'n_estimators', default=100, converter=int),
        max_features=get_param(best_params, 'model__max_features', 'max_features', default=1.0, converter=float),
        min_samples_split=get_param(best_params, 'model__min_samples_split', 'min_samples_split', default=2, converter=int),
        min_samples_leaf=get_param(best_params, 'model__min_samples_leaf', 'min_samples_leaf', default=1, converter=int)
    ))
])

print("Обучаем модель на всей выборке...")
final_model = final_pipeline.fit(X_full, y_full)
print("Модель успешно обучена на всех данных!")


Параметры лучшей модели: {'memory': 'None', 'steps': "[('preprocessor', ColumnTransformer(transformers=[('num', StandardScaler(),\n                                 ['battery_power', 'clock_speed', 'fc',\n                                  'int_memory', 'm_dep', 'mobile_wt', 'pc',\n                                  'px_height', 'px_width', 'ram', 'sc_h',\n                                  'sc_w', 'talk_time', 'battery_efficiency',\n                                  'screen_size']),\n                                ('cat', TargetEncoder(),\n                                 ['blue', 'dual_sim', 'four_g', 'three_g',\n                                  'touch_screen', 'wifi', 'n_cores'])])), ('model', RandomForestRegressor(random_state=42))]", 'transform_input': 'None', 'verbose': 'False', 'preprocessor': "ColumnTransformer(transformers=[('num', StandardScaler(),\n                                 ['battery_power', 'clock_speed', 'fc',\n                                  'int_memory', 'm_dep', 

In [33]:
final_predictions = final_model.predict(X_test)

cv_scores = cross_val_score(final_model, X_full, y_full, 
                           scoring='neg_mean_absolute_error', cv=5)
final_mae_cv = -np.mean(cv_scores)

metrics = {
    "MAE": mean_absolute_error(y_test, final_predictions),
    "MSE": mean_squared_error(y_test, final_predictions),
    "R2": r2_score(y_test, final_predictions)
}

print(f"CV MAE на всех данных: {final_mae_cv:.4f}")

experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name='Final_Production_Full_Data', experiment_id=experiment_id) as run:
    mlflow.sklearn.log_model(
        final_model,
        artifact_path="models",
        signature=signature,
        input_example=input_example,
        pip_requirements=req_file
    )
    
    
    mlflow.log_metrics(metrics)
    
    
    mlflow.log_param("MAE_cv", final_mae_cv)
    mlflow.log_param("MAE_cv_std", np.std(cv_scores))
    
    mlflow.log_params(best_params)
    mlflow.log_param("training_data", "full_dataset")
    mlflow.log_param("training_samples", len(X_full))
    mlflow.log_param("original_best_run", best_run_id)
    mlflow.log_param("model_type", "sequential_feature_selection")
    
    production_version = mlflow.register_model(
        f"runs:/{run.info.run_id}/models",  
        REGISTRY_MODEL_NAME,
        await_registration_for=300
    )
    
    client = mlflow.tracking.MlflowClient()  
    client.set_model_version_tag(
        name=REGISTRY_MODEL_NAME,
        version=production_version.version,
        key="stage",
        value="Production"
    )
    
    client.update_model_version(
        name=REGISTRY_MODEL_NAME,
        version=production_version.version,
        description=f"Final Production Model - Sequential Feature Selection trained on full dataset ({len(X_full)} samples)"
    )
    
    production_run_id = run.info.run_id
    print(f"Production модель зарегистрирована как версия {production_version.version}")

2025/11/27 05:17:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/27 05:17:22 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: matplotlib==3.10.6)
 - pandas (current: 2.2.2, required: pandas==2.3.2)
 - scikit-learn-1.7.2 (current: uninstalled, required: scikit-learn-1.7.2)
 - numpy (current: 2.0.2, required: numpy==1.26.4)
 - mlflow (current: 3.1.4, required: mlflow==2.16)
 - optuna-4.6.0 (current: uninstalled, required: optuna-4.6.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2025/11/27 05:17:22 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - matplotlib (current: 3.9.2, required: m

CV MAE на всех данных: 0.1556


Registered model 'price_range_prediction_model' already exists. Creating a new version of this model...
2025/11/27 05:17:22 WARNING mlflow.tracking._model_registry.fluent: Run with id adf10749f4b14844b47b31efe72d1370 has no artifacts at artifact path 'models', registering model based on models:/m-741c86c866d1420f878084332ee0b844 instead
2025/11/27 05:17:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: price_range_prediction_model, version 6
Created version '6' of model 'price_range_prediction_model'.


Production модель зарегистрирована как версия 6
🏃 View run Final_Production_Full_Data at: http://127.0.0.1:5000/#/experiments/5/runs/adf10749f4b14844b47b31efe72d1370
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/5


In [34]:
print(f"MAE для модели обученной на всей выборке: {metrics['MAE']:.2%}")
print(f"MSE для модели обученной на всей выборке: {metrics['MSE']:.2%}")
print(f"R2 для модели обученной на всей выборке: {metrics['R2']:.2%}")

MAE для модели обученной на всей выборке: 6.28%
MSE для модели обученной на всей выборке: 1.44%
R2 для модели обученной на всей выборке: 98.83%
